<h1><span style="background-color:blue; color:white; padding: 5px;">Notebook à executer lorsqu'il y a de nouvelles données disponible sur data.gouv.fr<br></h1>

<h2><span style="background-color:grey; color:white; padding: 5px;">Récupération des données disponible<br></h2>

In [ ]:
# Récupérer la date dans une variable + les données dans un dataframe

<h2><span style="background-color:grey; color:white; padding: 5px;">Nettoyage des données<br></h2>

In [ ]:
# Utiliser la même méthode que dans la première insertion de données effectuée dans la branche datagouv

<h2><span style="background-color:grey; color:white; padding: 5px;">Insertion des données<br></h2>

In [ ]:
# Récupérer la date la plus ancienne du dataframe
# Supprimer dans la bdd les données supérieure ou égale à la date stockée dans la variable du point1
# Insérer les données

<h2><span style="background-color:grey; color:white; padding: 5px;">Entraînement d'un modèle<br></h2>

In [ ]:
# Se baser sur les modèles créé dans mlflow et entraîner un modèle sur les mêmes paramètres
# Enregistrer dans mlflow, experiment : prod_{date_du_jour}

<h2><span style="background-color:grey; color:white; padding: 5px;">Créer un csv pour être utilisé dans streamlit<br></h2>

In [ ]:
# Pour éviter de trop requêter sur rds mais aussi avoir une execution rapide de l'application.
# Récupérer les données Region, département, communes, prix_M²/mois sur la dernière année ou 
# les deux dernières années suivant le modèle utilisé et par type de bien

<h2><span style="background-color:grey; color:white; padding: 5px;">Transformation du dataframe en csv et stockage dans AWS s3<br></h2>